In [1]:
# Li-Hames

In [ ]:
import math as m
import numpy as np
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import time
from copy import deepcopy
from numpy.linalg import matrix_power
from scipy.linalg import block_diag

In [ ]:
class Kite:
    def __init__(
        self, 
        u: List[float], #kite location
        v: List[float], #kite velocity
        f: float #f value
             ):
        self.u1 = u1
        self.u2 = u2
        self.u3 = u3
        self.v1 = v1
        self.v2 = v2
        self.v3 = v3
        self.f = f
        
class KiteSwarm:
    def __init__(
        self,
        swarm_size: int, #swarm size
        dim: int, #dimentions
        iter_max: int, #max number of iterations
        Nc: int, #control horizont
        Np: int, #horisont of prediction
        integr_steps: int, #number of integration steps --- (Nmax in article)
        step_size: float, #size of integration steps --- (h in article)
        closenes: float, #parameter of closenes --- (q in article)
        deleted_kites: float, #parameter of deleted kites --- (sigma in article)
        dist_min: float, #minimal distance from food to kite's leader
        k: float, #coeffitient of proportional navigation  
        area_u, #area for the control generation
        task_type_max, #min (sort False) or max (sort True)
    ):
        #_________PARAMETERS of ALG______
        self.swarm_size = swarm_size
        self.dim = dim
        self.u_dim = dim-1
        self.iter_max = iter_max
        self.Nc = Nc # Np have to be greater than Nc ! (do I need to check it?)
        self.Np = Np
        self.integr_steps = integr_steps
        self.step_size = step_size
        self.closenes = closenes
        self.deleted_kites = deleted_kites
        self.dist_min = dist_min
        self.k = k
        self.area_u = np.array(area_u)
        self.task_type_max = task_type_max
        
        self.population = [] #list of kites
        self.pool = []
        
        self.vector_r1 = [] # leader's location and velocity (const for current iter and  swarm)
        self.vector_x_ki1 = [] # vector for current kite at the moment t
        self.vector_x1 = [] #
        
        self.vector_r2 = [] # leader's location and velocity (const for current iter and  swarm)
        self.vector_x_ki2 = [] # vector for current kite at the moment t
        self.vector_x2 = [] #
        
        self.vector_r3 = [] # leader's location and velocity (const for current iter and  swarm)
        self.vector_x_ki3 = [] # vector for current kite at the moment t
        self.vector_x3 = [] #
        
        
        #_________KITES FOR STEP 3______
        best_kite1 = []
        leader_kite1 = []
        first_kite1 = []
        second_kite1 = []
        third_kite1 = []
        
        best_kite2 = []
        leader_kite2 = []
        first_kite2 = []
        second_kite2 = []
        third_kite2 = []
        
        best_kite3 = []
        leader_kite3 = []
        first_kite3 = []
        second_kite3 = []
        third_kite3 = []
        
        #_________PARAMETERS FOR CONTROL______
        self.best_x1 = []
        self.best_x2 = []
        self.best_x3 = []
        
        
        #_________MATRIX______
        self.E = np.eye(self.u_dim)
        
        self.hE =  self.step_size * self.E
        self.h2E = self.step_size**2 / 2 * self.E
        self.zero_mtrx = np.zeros((self.u_dim, self.u_dim))
        
        # init matrix A
        self.A = np.block([
            [self.E, self.hE], \
            [self.zero_mtrx, self.E]
        ])
        
        # init matrix B
        self. B = np.block([
            [self.h2E], \
            [self.hE]
        ])
        
        # init matrix B
        self.Q = self.closenes * np.eye(self.u_dim * self.Nc)
        
        tmp_A = np.block([
            [self.E, self.hE], \
            [self.zero_mtrx, self.E]
        ])
        
        # init matrix F
        A_list = [list(matrix_power(self.A,i+1)) for i in range(self.Np)]
        self.F = block_diag(*A_list)
        
        
        # init matrix Phi

        self.zero_B = np.zeros((2*self.u_dim, self.u_dim))
        
        list_columns = []
        
        # run through the columns
        for i in range(self.Np):
            # run through the rows in current column
            list_row = []
            for j in range(self.Nc):
                    
                if(j <= i):                        
                    list_row.append(matrix_power(self.A, (i-j)).dot(self.B))
                
                else:
                    list_row.append(self.zero_B)
            list_columns.append(list_row)
        self.Phi = np.block(list_columns)
        
    #_________________CALC FUNCTIONS_________________     
    def init_area(self, control):
        self.area_1 = [np.array([0, 4]) for i in range(self.u_dim)]
        self.area_2 = [np.array([0, 4]) for i in range(self.u_dim)]
        self.area_3 = [np.array([0, 0.5]) for i in range(self.u_dim)]
        self.area_new_1 = deepcopy(self.area_1)
        self.area_new_2 = deepcopy(self.area_2)
        self.area_new_3 = deepcopy(self.area_3)
        self.exact = control.ExactI()

    
    # creration a first population of kites 
    def form_first_pop(self, control):
        self.population.clear()
        for i in range(self.swarm_size):
            coordinate = np.array([np.random.rand() * (self.area[j][1] - self.area[j][0]) + self.area[j][0] for j in range(self.u_dim)])

            velocity = np.array([(np.random.rand() * (self.area[j][1] - self.area[j][0]) + self.area[j][0]) / self.integr_steps for j in range(self.u_dim)])
            
            member = Kite(coordinate, velocity, 0)
            self.population.append(member)
            
        
    
    def func_calculate(self, control):
        for i in range(self.swarm_size): 
            coordinate = self.population[i].u
            result = control.Functional(coordinate)
            self.population[i].f = result
            
    def func_barr1(self, step4, control):
        # определим списки
        self.vector_x.clear()
        vector_u = []
        kite_path_barr = [] # стая на каждом шаге
        kite_population_next_step = []
        best_in_each_track = []

        # перелет каждой особи
        for k_i in range(0, self.swarm_size):
            # очистим все списки
            vector_u = list(vector_u)
            vector_u.clear()

    #         kite.vector_x - список всех векторов состояний на горизонте 

            # временно определим vector_r, x_k_i 
            self.vector_x_ki = np.array([np.concatenate([self.population[k_i].u, self.population[k_i].v]) for i in range(self.Np)])

            # deepcopy R, X_ki для мат операций
            tmp_r = deepcopy(self.vector_r)
            tmp_x_ki = deepcopy(self.vector_x_ki)


            # reshape R, X_ki для мат операций
            tmp_r = np.array(tmp_r).reshape((2 * self.u_dim * self.Np, 1))
            tmp_x_ki = np.array(tmp_x_ki).reshape((2 * self.u_dim * self.Np, 1))


            # --------------ОПРЕДЕЛЕНИЕ ВЕКТОРА U + reshape--------------
            u = np.linalg.inv(self.Phi.transpose().dot(self.Phi) + self.Q).dot(self.Phi.transpose()).dot(tmp_r - self.F.dot(tmp_x_ki))
            vector_u.append(u) #зачем аппенд если можно сразу преобразовать в вектор? 

            vector_u = np.block(vector_u) #чтобы из списка преобразовать в матрицу?
            vector_u = vector_u.reshape((self.u_dim, self.Nc)).transpose()
    #         print('u', '\n', vector_u)


            # --------------ОПРЕДЕЛЕНИЕ ВЕКТОРА U  для шага 2.1 или 4--------------
            if(step4 == 0):
                temp = np.zeros((self.Nc, self.Nc))
                temp[0, 0] = 1.0
                u = temp.dot(vector_u)
    #             print('u', '\n', u)


            kite_path_barr.clear()

            # --------------ВЫЧИСЛЕНИЕ НОВЫХ КООРДИНАТ (ТЕКУЩЕГО КОРШУНА) ВЕКТОРА X_ki--------------
            # когда Nc (Nc<Np) заканчивается, u определяем как [0., 0.]
            for i in range(self.Np):

                if(i < len(vector_u)):
                    u_i = vector_u[i]
                else:
                    u_i = np.zeros(self.u_dim)

                tmp_x = self.A.dot(self.vector_x_ki[i]) + self.B.dot(u_i) 
                coordinate = tmp_x[:self.u_dim]
                velocity = tmp_x[self.u_dim: self.u_dim*2+1]


                # проверка на выход за границы, если вышли - берем крайнее значение
                for j in range(len(coordinate)):
                    if coordinate[j] < self.area[j][0]:
    #                     print('вылетели 1')
                        coordinate[j] = self.area[j][0]
                    elif coordinate[j] > self.area[j][1]:
    #                     print('вылетели 2')
                        coordinate[j] = self.area[j][1]

                    if velocity[j] < self.area[j][0]:
    #                     print('вылетели 3')
                        velocity[j] = self.area[j][0]
                    elif velocity[j] > self.area[j][1]:
    #                     print('вылетели 4')
                        velocity[j] = self.area[j][1]

                result = control.Functional(coordinate)
                member = Kite(coordinate, velocity, result)
                kite_path_barr.append(member) #записываем сюда весь трэк коршуна, чтобы потом выбрать конкретное положение

            self.vector_x.append(kite_path_barr)

            kite_pb = deepcopy(kite_path_barr)
            kite_pb.sort(reverse=self.task_type_max, key=lambda kite_: kite_.f)
    #         kite_population_next_step.append(deepcopy(kite_path_barr[kite.Np-1]))
            kite_population_next_step.append(deepcopy(kite_pb[0]))

        self.population.clear()
        self.population = deepcopy(kite_population_next_step)
        kite_population_next_step.clear()
    
    
    # step_size: float, #size of integration steps --- (h = 2 in article)
    def move_kite_1(self, control):
        u_step3 = []
        T = 10
        i = 0
        for kite_01 in self.population[0:]:
            firts = 6 * (self.leader_kite.u - kite_01.u) / ((self.swarm_size - i)**2 * self.step_size**2)
            second = 4 * kite_01.v / ((self.swarm_size - i) * self.step_size)
            ++i
            result = firts - second
            u_step3.append(result)
        s = np.array([np.concatenate([self.population[i].u, self.population[i].v]) for i in range(self.swarm_size)])
        s = np.transpose(s)
        u_step3 = np.transpose(u_step3)
        x = self.A.dot(s) + self.B.dot(u_step3)
        x = np.transpose(x)
        list1 = []
        for ii in range(self.swarm_size):
            coordinate = x[ii][:self.u_dim]
            velocity = x[ii][self.u_dim: self.u_dim*2+1]
            result = control.Functional(coordinate)
            member = Kite(coordinate, velocity, result)
            list1.append(member)

        list1.sort(reverse=self.task_type_max, key=lambda kite_: kite_.f)
        self.population[1] = list1[1]
        self.best_kite.append(list1[1])


    def delete_kites(self, control):
        # deleted_kites: float, #parameter of deleted kites --- (sigma in article)
        number_of_deleted = int(round(self.deleted_kites * self.swarm_size / 100, 0))
        for i in range(number_of_deleted):
            if(len(self.population) != 0):
                self.population.pop()
        for i in range(number_of_deleted):
            coordinate = np.array([np.random.rand() * \
                                (self.area[j][1] - self.area[j][0]) + self.area[j][0] \
                                  for j in range(self.u_dim)])

            velocity = np.array([(np.random.rand() * \
                        (self.area[j][1] - self.area[j][0]) + self.area[j][0]) / self.integr_steps \
                                  for j in range(self.u_dim)])

            result = control.Functional(coordinate)
            member = Kite(coordinate, velocity, result)
            self.population.append(member)
            
            
    #_________________WORK FUNCTION_________________         
    def kso_work(self, control):
        start = time.monotonic()
        time.sleep(0.1)
        self.init_area(control)
        self.population.clear()
        self.form_first_pop(control)
        self.func_calculate(control)

        self.population.sort(reverse=self.task_type_max, key=lambda kite_: kite_.f)
        #self.pool.append(self.population[0])

        # --------- шаг 2 --------- (добавить цикл iter_max)

        self.vector_r = np.array([np.concatenate([self.population[0].u, self.population[0].v]) for i in range(self.Np)])
        for j in range(self.iter_max): 
            for i in range(self.Nc):
                self.func_barr1(1, control)
            self.population.sort(reverse=self.task_type_max, key=lambda kite_: kite_.f)
            self.pool.append(self.population[0])
            self.vector_r = np.array([np.concatenate([self.population[0].u, self.population[0].v]) for i in range(self.Np)])

            # --------- шаг 3 ---------

            self.leader_kite = self.population[0]
            self.leader_kite.v = np.zeros(self.u_dim)
            self.first_kite = self.population[1]
            self.second_kite = self.population[2]
            self.third_kite = self.population[3]
            self.best_kite = []
            self.move_kite_1(control)
            
            self.population.sort(reverse=self.task_type_max, key=lambda kite_: kite_.f)
            self.pool.append(self.population[0])

            # --------- шаг 4 ---------

            self.func_barr1(0, control)
            self.population.sort(reverse=self.task_type_max, key=lambda kite_: kite_.f)
            self.pool.append(self.population[0])

            # --------- шаг 5 ---------
            self.delete_kites(control) 
            self.population.sort(reverse=self.task_type_max, key=lambda kite_: kite_.f)
            self.pool.append(self.population[0])

        self.pool.sort(reverse=self.task_type_max, key=lambda kite_: kite_.f)
        self.MOMENT = 1
        end = time.monotonic()
        self.make_best_x(self.pool[0].u, control.best_x1x2)
        self.draw_plots()


        print('work time : {:>9.2f}'.format(end-start))
        print('Approximate', self.pool[0].f,'\nPresize = ', self.exact,'\nError = ', (self.exact - self.pool[0].f))

    
    #_________________DRAWING_________________  
    def make_best_x(self, u1, u2, u3):
        self.best_x1.clear()
        self.best_x2.clear()
        self.best_x3.clear()
        self.best_x1.append(2)
        self.best_x2.append(5)
        self.best_x3.append(7)
        
        for i in range(1, self.dim):
            x1_temp = self.best_x1[i - 1] / (1 + 0.01 * u1[i - 1] * (3 + u2[i - 1]))
            x2_temp = (self.best_x2[i - 1] + u1[i - 1] * x1_temp) / (1 + u1[i - 1] * (1 + u2[i - 1]))
            x3_temp = self.best_x3[i - 1] / (1 + 0.01 * u1[i - 1] * (1 + u3[i - 1]))
            self.best_x1.append(x1_temp)
            self.best_x2.append(x2_temp)
            self.best_x3.append(x3_temp)
        
        self.best_x1 = np.array(self.best_x1)
        self.best_x2 = np.array(self.best_x2)
        self.best_x3 = np.array(self.best_x3)
    
    def draw_plots(self):
        plt.figure(figsize=(18, 10))
        
        plt.subplot(2,3,1)
        plt.plot(self.pool[0].u1, '.')
        plt.ylabel('u1')
        plt.xlabel('t')
        
        plt.subplot(2,3,2)
        plt.plot(self.pool[0].u2, '.')
        plt.ylabel('u2')
        plt.xlabel('t')
        
        plt.subplot(2,3,3)
        plt.plot(self.pool[0].u3, '.')
        plt.ylabel('u3')
        plt.xlabel('t')

        plt.subplot(2,3,4)
        plt.plot(self.best_x1, '.')
        plt.ylabel('$x_{1}$')
        plt.xlabel('t')

        plt.subplot(2,3,5)
        plt.plot(self.best_x2, '.')
        plt.ylabel('$x_{2}$')
        plt.xlabel('t')
        
        plt.subplot(2,3,6)
        plt.plot(self.best_x3, '.')
        plt.ylabel('$x_{3}$')
        plt.xlabel('t')

        plt.show()
            

In [ ]:
class LuusTassonTask:
    def __init__(
    self, 
    N_step: int,
    ):
        self.N_step = N_step # N is the dimention of the x, dim for u is the N-1

    def make_x(self, u1, u2, u3):
        x1 = []
        x2 = []
        x3 = []
        x1.append(2)
        x2.append(5)
        x3.append(7)
        for i in range(1, self.N_step):
            x1_temp = x1[i - 1] / (1 + 0.01 * u1[i - 1] * (3 + u2[i - 1]))
            x2_temp = (x2[i - 1] + u1[i - 1] * x1_temp) / (1 + u1[i - 1] * (1 + u2[i - 1]))
            x3_temp = x3[i - 1] / (1 + 0.01 * u1[i - 1] * (1 + u3[i - 1]))
            x1.append(x1_temp)
            x2.append(x2_temp)
            x3.append(x3_temp)
        x1 = np.array(x1)
        x2 = np.array(x2)
        x3 = np.array(x3)
        return x1, x2, x3

    def Functional(self, u1, u2, u3):
        x1 = []
        x2 = []
        x3 = []
        u1 = np.array(u1)
        u2 = np.array(u2)
        u3 = np.array(u3)
        x1, x2, x3 = self.make_x(u1, u2, u3)
        N = self.N_step - 1

        
        temp_1 = 0
        for i in range(1, self.N_step):
            temp_1 += (x1[i - 1]**2 + x2[i - 1]**2 + 2 * u3[i - 1]**2)
        
        temp_2 = 0
        for i in range(1, self.N_step):
            temp_2 += (x3[i - 1]**2 + 2 * u1[i - 1]**2 + 2 * u2[i - 1]**2)
            
        temp = x1[N]**2 + x2[N]**2 + x3[N]**2 + (temp_1 * temp_2)**0.5
        return temp
        
        
    def ExactI(self):
        if self.N_step == 100:
            return 258.3393

In [ ]:
a = LuusTassonTask(100)
kite = KiteSwarm(
    15, #swarm_size
    b.N_step, #dim
    20, #iter_max
    15, #Nc
    30, #Np
    0.07, #integr_steps
    0.3, #step_size
    1, #closenes
    20, #deleted_kites
    1, #dist_min
    3, #k
    [0, 1],#area
    True) #max 
kite.kso_work(a)

In [ ]:
def mtx_E(self):
            return np.eye(self.u_dim)
        
        def mtx_hE(self, mtxE):
            return self.step_size * mtxE
        
        def mtx_h2E(self, mtxE):
            return self.step_size**2 / 2 * mtxE
        
        def mtx_O(self):
            return np.zeros((self.u_dim, self.u_dim))
        
        def mtx_A(self, mthE, mtxhE, mtxO):
            return np.block([[mthE, mtxhE], [mtxO, mthE]])
        
        def mtx_B(self, mtxh2E, mtxhE):
            return np.block([[mtxh2E], [mtxhE]])
        
        def mtx_Q(self):
            return self.closenes * np.eye(self.u_dim * self.Nc)
        
        def mtx_F(self, mtxA):
            A_list = [list(matrix_power(mtxA, i+1)) for i in range(self.Np)]
            F = block_diag(*A_list)
            return F
        
        def mtx_Phi(self, mtxA, mtxB):
            zero_B = np.zeros((2*self.u_dim, self.u_dim))
            list_columns = []
            # run through the columns
            for i in range(self.Np):
                # run through the rows in current column
                list_row = []
                for j in range(self.Nc):

                    if(j <= i):                        
                        list_row.append(matrix_power(mtxA, (i-j)).dot(mtxB))

                    else:
                        list_row.append(zero_B)
                list_columns.append(list_row)
            Phi = np.block(list_columns)
            return Phi